In [1]:
import pandas as pd
df = pd.read_csv('../data/hansard_speeches_processed.csv')
df = df.sample(n=50000, random_state = 42)

import sys
sys.path.append('../gtm/')
from corpus import GTMCorpus
from gtm import GTM

# Create a GTMCorpus object
train_dataset = GTMCorpus(
    df, 
    embeddings_type = None,
    prevalence = "~ party", 
    content = "~ party"
)

# Train the model
tm = GTM(
    train_dataset, 
    doc_topic_prior='dirichlet',
    update_prior=True,
    decoder_type='sage',
    num_epochs=10
)

/home/germain/Documents/topic models/generalized_topic_model/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch   1	Iter   10	Loss:622.4044800	Rec Loss:615.2288208	MMD:7.1755919	Sparsity_Loss:0.0000446	Pred_Loss:0.0000000
Epoch   1	Iter   20	Loss:717.2976685	Rec Loss:713.5053711	MMD:3.7922425	Sparsity_Loss:0.0000447	Pred_Loss:0.0000000
Epoch   1	Iter   30	Loss:629.0126953	Rec Loss:626.1434326	MMD:2.8691957	Sparsity_Loss:0.0000449	Pred_Loss:0.0000000
Epoch   1	Iter   40	Loss:627.8161011	Rec Loss:624.9390869	MMD:2.8769379	Sparsity_Loss:0.0000451	Pred_Loss:0.0000000
Epoch   1	Iter   50	Loss:662.2515259	Rec Loss:659.4042358	MMD:2.8472276	Sparsity_Loss:0.0000453	Pred_Loss:0.0000000
Epoch   1	Iter   60	Loss:661.3048706	Rec Loss:658.5788574	MMD:2.7259717	Sparsity_Loss:0.0000456	Pred_Loss:0.0000000
Epoch   1	Iter   70	Loss:584.1768799	Rec Loss:582.2224121	MMD:1.9544032	Sparsity_Loss:0.0000460	Pred_Loss:0.0000000
Epoch   1	Iter   80	Loss:563.3456421	Rec Loss:560.2634277	MMD:3.0821428	Sparsity_Loss:0.0000463	Pred_Loss:0.0000000
Epoch   1	Iter   90	Loss:678.2570190	Rec Loss:675.8744507	MMD:2.3825226	

Epoch   4	Iter  150	Loss:628.8065186	Rec Loss:628.4115601	MMD:0.3897625	Sparsity_Loss:0.0051591	Pred_Loss:0.0000000
Epoch   4	Iter  160	Loss:596.8493042	Rec Loss:596.5529785	MMD:0.2910149	Sparsity_Loss:0.0053240	Pred_Loss:0.0000000
Epoch   4	Iter  170	Loss:577.3053589	Rec Loss:576.4865723	MMD:0.8132652	Sparsity_Loss:0.0055086	Pred_Loss:0.0000000
Epoch   4	Iter  180	Loss:517.1914673	Rec Loss:516.7437744	MMD:0.4420264	Sparsity_Loss:0.0056689	Pred_Loss:0.0000000
Epoch   4	Iter  190	Loss:657.3010864	Rec Loss:656.7469482	MMD:0.5482616	Sparsity_Loss:0.0058296	Pred_Loss:0.0000000
Epoch   5	Iter   10	Loss:618.9647827	Rec Loss:618.4434814	MMD:0.5184423	Sparsity_Loss:0.0028419	Pred_Loss:0.0000000
Epoch   5	Iter   20	Loss:548.0536499	Rec Loss:546.8892822	MMD:1.1613188	Sparsity_Loss:0.0030443	Pred_Loss:0.0000000
Epoch   5	Iter   30	Loss:619.2995605	Rec Loss:618.3542480	MMD:0.9420116	Sparsity_Loss:0.0032662	Pred_Loss:0.0000000
Epoch   5	Iter   40	Loss:569.3580322	Rec Loss:568.7135010	MMD:0.6411164	

Epoch   7	Iter  140	Loss:636.1221313	Rec Loss:634.4176025	MMD:1.6997645	Sparsity_Loss:0.0047794	Pred_Loss:0.0000000
Epoch   7	Iter  150	Loss:572.1350098	Rec Loss:570.2172852	MMD:1.9127916	Sparsity_Loss:0.0049175	Pred_Loss:0.0000000
Epoch   7	Iter  160	Loss:617.8211670	Rec Loss:615.4873047	MMD:2.3287873	Sparsity_Loss:0.0050437	Pred_Loss:0.0000000
Epoch   7	Iter  170	Loss:648.1019287	Rec Loss:646.2041626	MMD:1.8925390	Sparsity_Loss:0.0052228	Pred_Loss:0.0000000
Epoch   7	Iter  180	Loss:600.9811401	Rec Loss:599.3749390	MMD:1.6007940	Sparsity_Loss:0.0054175	Pred_Loss:0.0000000
Epoch   7	Iter  190	Loss:591.2868042	Rec Loss:589.3916016	MMD:1.8896136	Sparsity_Loss:0.0055895	Pred_Loss:0.0000000
Epoch   8	Iter   10	Loss:504.8130493	Rec Loss:502.5690918	MMD:2.2411361	Sparsity_Loss:0.0028108	Pred_Loss:0.0000000
Epoch   8	Iter   20	Loss:591.8581543	Rec Loss:589.4499512	MMD:2.4050958	Sparsity_Loss:0.0031131	Pred_Loss:0.0000000
Epoch   8	Iter   30	Loss:627.4268188	Rec Loss:625.3436279	MMD:2.0798683	

In [2]:
# Most common words
import torch
from torch.nn import functional as F
num_words = 8
gamma = tm.AutoEncoder.beta.bias
top_words = torch.topk(gamma, k=num_words, dim=0).indices.cpu().numpy().flatten()
[tm.id2token[a] for a in top_words]

['hon', 'people', 'make', 'friend', 'say', 'year', 'need', 'take']

In [4]:
# Labour Party deviation (for the whole corpus)
from torch.nn import functional as F
gamma = tm.AutoEncoder.beta_c.weight[:,1:2]
top_words = torch.topk(gamma, k=num_words, dim=0).indices.cpu().numpy().flatten()
[tm.id2token[a] for a in top_words]

['amputate',
 'bait',
 '99',
 'personalisation',
 'heir',
 'vista',
 'lavish',
 'bequest']

In [5]:
# Labour Party deviation (for each topic)
for k in range(20):
    gamma = tm.AutoEncoder.beta_ci.weight[:,k*2+1]
    top_words = torch.topk(gamma, k=num_words, dim=0).indices.cpu().numpy().flatten()
    print([tm.id2token[a] for a in top_words])

['agree', 'gentleman', 'welcome', 'millennium', 'assure', 'cut', 'hon', 'friend']
['order', 'ask', 'statement', 'issue', 'point', 'answer', 'question', 'information']
['vote', 'favour', 'democracy', 'elect', 'politic', 'hold', 'representative', 'agreement']
['health', 'hospital', 'disability', 'service', 'agree', 'friend', 'visit', 'ensure']
['cut', 'wage', 'vat', 'recession', 'million', '50p', 'scrap', 'hurt']
['friend', 'agree', 'gentleman', 'point', 'assure', 'answer', 'police', 'decision']
['table', 'order', 'amendment', 'disagree', 'appointment', 'draft', 'peer', 'power']
['peace', 'resolution', 'unionist', 'war', 'agreement', 'bombing', 'violence', 'security']
['gentleman', 'accept', 'statement', 'point', 'agree', 'mislead', 'order', 'answer']
['poverty', 'cut', 'week', 'winter', 'meal', 'housing', 'wage', 'family']
['school', 'child', 'parent', 'learning', 'education', 'foundation', 'disability', 'university']
['decision', 'gentleman', 'friend', 'point', 'answer', 'welcome', 'as